In [ ]:
import os
import re

import pandas as pd
import numpy as np

import matplotlib
from matplotlib import pyplot as plt
%matplotlib notebook

from parsers import generic, dlaf, slate, dplasma, scalapack

colors_default = plt.cm.get_cmap('tab10').colors

LIB_COLOR = {
    'dlaf': colors_default[2], 
    'slate': colors_default[0],
    'dplasma': colors_default[1],
    'scalapack': colors_default[4],
    'prototype': 'k',
}

## Collect data from filesystem

In [ ]:
slurm_db = {}
for slurm_db_filepath in [
    'slurm.db',
    '/Users/ialberto/workspace/benchmarks/20200619-benchmark-hpxmod_and_mpi-executor/raw_data/slurm.db',
        ]:
    data = generic.parse_slurm_db(slurm_db_filepath)
    slurm_db = {**slurm_db, **data}

In [ ]:
campaigns = {
    'post': 'raw_data/post/cholesky/',
    '1.5.0': 'raw_data/porting',
#    'ref': '/Users/ialberto/workspace/benchmarks/20200619-benchmark-hpxmod_and_mpi-executor/raw_data',
    'ref-mar20': '/Users/ialberto/workspace/benchmarks/20200317-DLAF_miniappcholesky-pre/raw_data',
}

jobs_reports = []

df = pd.DataFrame([], columns=[
    'library',
    'benchmark_lib',
    'run_index',
    'm',
    'mb',
    'grid_rows',
    'grid_cols',
    'time',
    'performance',
    'id',
    'campaign',
])

for campaign_name, workdir in campaigns.items():
    for entry in os.listdir(workdir):
        m = re.match('(\d+)J\d+_\d+', entry)

        if not m:
            continue

        jobid, = m.groups()
        jobdata = slurm_db[jobid]
        jobname = jobdata['jobname']

        first_seen = jobid not in jobs_reports
        jobs_reports.append(jobid)

        if (jobdata['state'] != 'COMPLETED'):
            if first_seen:
                display(f'WARNING {jobid} {jobname} {jobdata["state"]}')
            continue

        # select parser
        if re.search(r'.*slate.*', jobname):
            benchmark_lib = 'slate'
            parser = slate.parse_cholesky
        elif re.search(r'.*dplasma.*', jobname):
            benchmark_lib = 'dplasma'
            parser = dplasma.parse_cholesky
        elif re.search(r'.*scalapack.*', jobname):
            benchmark_lib = 'scalapack'
            parser = scalapack.parse_cholesky
        else:
            benchmark_lib = 'dlaf'
            parser = dlaf.parse_cholesky

        current_run = os.path.join(workdir, entry)
        for task_entry in os.listdir(current_run):
            if not task_entry.endswith('.out'):
                continue

            out_filepath = os.path.join(current_run, task_entry)
            try:
                current_data = parser(out_filepath)
            except Exception as e:
                display(f'ERROR {out_filepath} {e}')
                continue

            if len(current_data[1]) == 0:
                display(f'No data in {out_filepath}')
                continue

            current_df = pd.DataFrame(current_data[1], columns=[
                'run_index',
                'm',
                'mb',
                'grid_rows',
                'grid_cols',
                'time',
                'performance',
            ])

            # do not plot results with more than 64 nodes
            if (jobdata["nodes"] > 64):
                continue

            current_df['campaign'] = campaign_name
            current_df['library'] = benchmark_lib
            current_df['nodes'] = jobdata["nodes"]
            current_df['ranks_per_node'] = current_df['grid_rows'] * current_df['grid_cols'] / current_df["nodes"]
            current_df['benchmark_lib'] = benchmark_lib + "|" + str(int(current_df['ranks_per_node'].iloc[0]))
            current_df['performance_per_node'] = current_df['performance'] / current_df['nodes']

            current_df['id'] = entry

            df = df.append(current_df)

display("LOADED")

In [ ]:
df

In [ ]:
df_runs = df\
.loc[df['run_index'] != 0]\
.groupby(['library', 'm', 'mb', 'nodes', 'ranks_per_node', 'benchmark_lib', 'campaign'])\
.agg(
    p_mean=("performance", 'mean'),
    p_min=("performance", 'min'),
    p_max=("performance", 'max'),
    ppn_mean=("performance_per_node", 'mean'),
    ppn_min=("performance_per_node", 'min'),
    ppn_max=("performance_per_node", 'max'),
    time_mean=("time", 'mean'),
    measures=('performance', 'count')
)\
.reset_index()

display(df_runs)

In [ ]:
df_groups = df_runs.groupby(['m', 'mb'])

domain_m = df['m'].unique().tolist()
domain_mb = df['mb'].unique().tolist()

domain_m.sort()
domain_mb.sort()

fig, axs_perf = plt.subplots(len(domain_mb), len(domain_m),
                             squeeze=False, sharey='col', figsize=(9,12))

for (m, mb), grp_data in df_groups:
    ax_i, ax_j = domain_mb.index(mb), domain_m.index(m)
    ax_perf = axs_perf[ax_i, ax_j]
    
    for (lib, campaign), lib_data in grp_data.groupby(['benchmark_lib', 'campaign']):
        lib_name, lib_variant = lib.split("|")
        lib_data.plot(ax=ax_perf, x='nodes', y='ppn_mean',
                      marker='.', linestyle = '-' if lib_variant == '1' else '--',
                      label=f'{lib}@{campaign}', color=LIB_COLOR[lib_name])

    ax_perf.set_ylim(0, 1200)
    if ax_j == 0:
        ax_perf.set_ylabel(f'GFlops/node\n(mb={mb})')
    
    for ax in [ax_perf]:
        ax.get_legend().remove() #loc="upper right")
        ax.set_xscale('log', basex=2)
        ax.set_xticks([2**x for x in range(7)])
        ax.grid(color='gray', alpha=0.3)
        
        if ax_i == len(domain_mb) - 1:
            ax.set_xlabel(f'nodes')
        if ax_i == 0:
            ax.set_title(f'm={m}')

In [ ]:
df_runs_multi = df_runs.set_index(['library', 'ranks_per_node', 'mb', 'campaign']).sort_index()

display(df_runs_multi)

### Manually add data from HPX-prototype

In [ ]:
df_prototype = pd.read_csv(
    '/Users/ialberto/workspace/benchmarks/20200317-DLAF_miniappcholesky-pre/res/resgf_hpx-mc.txt',
    header=0,
    index_col=False,
    names=[
        'm', 'mb',
        'nodes', 'ranks_per_node', 'cores_per_task',
        'time_mean', 'time_min',
        'p_mean', 'p_max',
        'ppn_mean', 'ppn_max'],
    )

df_prototype['library'] = 'prototype'
df_prototype['benchmark_lib'] = 'prototype|1'
df_prototype['campaign'] = 'ref'

df_prototype.set_index(['library', 'ranks_per_node', 'mb', 'campaign'], inplace=True)
df_prototype = df_prototype[df_prototype.nodes <= 2**6]

df_prototype = df_prototype.drop(index=('prototype', 1, 256, 'ref')).sort_index()

df_runs_multi = df_runs_multi.append(df_prototype).sort_index()
# display(df_runs_multi[df_runs_multi.benchmark_lib == 'prototype|1'])

In [ ]:
def plot_interesting(interesting_configs, plot_time=False):
    if type(interesting_configs) == list:
        interesting_configs = {k:{} for k in interesting_configs}
    
    domain_m = df['m'].unique().tolist()
    domain_mb = df['mb'].unique().tolist()

    domain_m.sort()
    domain_mb.sort()

    fig, axs_perf = plt.subplots(len(domain_m), 1,
                                 squeeze=False, sharey='row', figsize=(9, 12))
    
    if plot_time:
        fig, axs_time = plt.subplots(len(domain_m), 1,
                                     squeeze=False, figsize=(9, 12))
        

    for config in interesting_configs.keys():
        plot_properties = {
            # default properties
            **dict(color=LIB_COLOR[config[0]], marker='.', linestyle = '-'),
            # passed-by-argument
            **interesting_configs[config]
        }
        mb = config[2]
        for m, grp_data in df_runs_multi.loc[config, :].groupby('m'):
            ax_i, ax_j = domain_m.index(m), 0
            ax_perf = axs_perf[ax_i, ax_j]
            
            if plot_time:
                ax_time = axs_time[ax_i, ax_j]

            for (lib, campaign_name), lib_data in grp_data.groupby(['benchmark_lib', 'campaign']):
                lib_name, lib_variant = lib.split("|")
                lib_data.plot(ax=ax_perf, x='nodes', y='ppn_mean',
                              label=config if len(config) > 3 else f'{config} {campaign_name}',
                              **plot_properties)
                ax_perf.fill_between(lib_data['nodes'], lib_data['ppn_min'], lib_data['ppn_max'],
                                     alpha=0.2, color=plot_properties['color'])
                
                if plot_time:
                    lib_data.plot(ax=ax_time, x='nodes', y='time_mean',
                                  label=config if len(config) > 3 else f'{config} {campaign_name}',
                                  **plot_properties)

            axs = [ax_perf]
                    
            ax_perf.set_ylabel(f'GFlops/node (with m={m})')
            ax_perf.set_ylim(0, 1200)
            
            if plot_time:
                axs.append(ax_time)
                ax_time.set_ylabel(f'Time (s)\n(with m={m})')

            for ax in axs:
                ax.legend(loc="upper right")
                ax.set_xscale('log', basex=2)
                ax.set_xticks([2**x for x in range(7)])
                ax.set_xlabel(f'nodes')
                ax.grid(color='gray', alpha=0.3)

In [ ]:
def plot_interesting_pres(m, interesting_configs, plot_time=False):
    if type(interesting_configs) == list:
        interesting_configs = {k:{} for k in interesting_configs}

    fig, axs_perf = plt.subplots(1, 1,
                                 squeeze=False, sharey='row', figsize=(4.5, 3.8))
    
    if plot_time:
        fig, axs_time = plt.subplots(1, 1,
                                     squeeze=False, figsize=(4.5, 3.8))
        

    for config in interesting_configs.keys():
        plot_properties = {
            # default properties
            **dict(color=LIB_COLOR[config[0]], marker='.', linestyle = '-'),
            # passed-by-argument
            **interesting_configs[config]
        }
        mb = config[2]
        for current_m, grp_data in df_runs_multi.loc[config, :].groupby('m'):
            if m != current_m:
                continue
            
            ax_i, ax_j = 0, 0
            ax_perf = axs_perf[ax_i, ax_j]
            
            if plot_time:
                ax_time = axs_time[ax_i, ax_j]

            for (lib, campaign_name), lib_data in grp_data.groupby(['benchmark_lib', 'campaign']):
                lib_name, lib_variant = lib.split("|")
                lib_data.plot(ax=ax_perf, x='nodes', y='ppn_mean', **plot_properties)
                ax_perf.fill_between(lib_data['nodes'], lib_data['ppn_min'], lib_data['ppn_max'],
                                     alpha=0.2, color=plot_properties['color'])
                
                if plot_time:
                    lib_data.plot(ax=ax_time, x='nodes', y='time_mean', **plot_properties)

            axs = [ax_perf]
                    
            ax_perf.set_ylabel(f'GFlops/node')
            ax_perf.set_ylim(0, 1200)
            
            if plot_time:
                axs.append(ax_time)
                ax_time.set_ylabel(f'Time (s)')

            for ax in axs:
                ax.legend(loc="upper right")
                ax.set_xscale('log', basex=2)
                ax.set_xticks([2**x for x in range(7)])
                ax.set_xlabel(f'nodes')
                ax.grid(color='gray', alpha=0.3)
                ax.set_title(f'Matrix Size = {m}')

### No Regression

In [ ]:
for mb in [256, 512]:
    plot_interesting({
      ('dlaf', 2, mb, 'post'): dict(linestyle='--', color='black'),
      ('dlaf', 2, mb,  '1.5.0'): {},
    }, True)

In [ ]:
for mb in [256, 512]:
    for campaign_vs in ['post', '1.5.0']:
        plot_interesting({
          ('dlaf', 2, mb, 'ref-mar20'): dict(linestyle='--', color='black'),
          ('dlaf', 2, mb,  campaign_vs): {},
        })

### Compare libraries (best configurations)

In [ ]:
plot_interesting([
    ('dlaf', 2, 512, 'post'),
    ('slate', 2, 384),
    ('dplasma', 1, 384),
    ('scalapack', 36, 192),
    ('prototype', 1, 512),
], True)

In [ ]:
plot_interesting_pres(20480, {
    ('dlaf', 2, 512, 'post'): dict(label='DLAF'),
    ('slate', 2, 384): dict(label='SLATE'),
    ('dplasma', 1, 384): dict(label='DPlasma'),
    ('scalapack', 36, 192): dict(label='ScaLAPACK (libsci)'),
    ('prototype', 1, 512): dict(label='Prototype'),
}, True)

In [ ]:
plot_interesting_pres(40960, {
    ('dlaf', 2, 512, 'post'): dict(label='DLAF'),
    ('slate', 2, 384): dict(label='SLATE'),
    ('dplasma', 1, 384): dict(label='DPlasma'),
    ('scalapack', 36, 192): dict(label='ScaLAPACK (libsci)'),
    ('prototype', 1, 512): dict(label='Prototype'),
}, True)

### Blocksizes

In [ ]:
MB_COLORS = [
    dict(color='red'),
    dict(color='orange'),
    dict(color='yellow'),
    dict(color='green'),
    dict(color='cyan'),
]

def plot_mbs(mbs_library):
    plot_interesting({k:v for k,v in zip(mbs_library, MB_COLORS)}, True)

In [ ]:
for rank_per_node in [1,2]:
    mbs = [('dlaf', rank_per_node, mb, 'post') for mb in df[df['library'] == 'dlaf'].mb.sort_values().unique().tolist()]
    plot_mbs(mbs)

In [ ]:
mbs = [('dplasma', 1, mb) for mb in df[df['library'] == 'dplasma'].mb.sort_values().unique().tolist()]

plot_mbs(mbs)

In [ ]:
for rank_per_node in [1,2]:
    mbs = [('slate', rank_per_node, mb) for mb in df[df['library'] == 'slate'].mb.sort_values().unique().tolist()]
    plot_mbs(mbs)

In [ ]:
mbs = [('scalapack', 36, mb) for mb in df[df['library'] == 'scalapack'].mb.sort_values().unique().tolist()]

plot_mbs(mbs)

### Comparison library configurations

In [ ]:
plot_interesting({
    ('dlaf', 1, 512, 'post'): {},
    ('dlaf', 2, 512, 'post'): dict(linestyle='--'),
}, True)

In [ ]:
plot_interesting({
    ('slate', 1, 384, 'post'): {},
    ('slate', 2, 384, 'post'): dict(linestyle='--'),
}, True)